In [1]:
import numpy as np
from myur import MyUR3e

robot = MyUR3e()
robot.move_gripper(255, 255, 255)
robot.clear_sim()

In [23]:
# LOCATION VARIABLES
HOME = [0.3, 0.3, 0.3, 0, 0, 0]
CAKE_CENTER = [0.1, 0.3, 0.32]
CAKE_RADIUS = 0.026
CAKE_HEIGHT = 0.075
LOWEST_Z = 0.207 # true value #0.25 for testing

# FROSTING VARIABLES
FROSTING_HALFWAY_JOINTS = [4.13, -1.88, -1.56, -1.55, 0.67, -2.88]
FROSTING_START_JOINTS = [3.46, -2.02, -2.36, -1.89, -1.21, 3.16]
FROSTING_LENGTH = 0.15
FROSTING_DROP_HEIGHT = 0.006
NUM_SCOOP = 0
FROSTING_SEGMENT = 2

def angle_clean(angle):
    angle = angle % 360
    if angle == 0:
        angle = 0.1
    return angle

def arc(divisor, segment=0, steps=100, center=CAKE_CENTER):
    # segment should be in range(divisor)
    # i.e. if divisor = 5, segment should be 0, 1, 2, 3, or 4

    angles = []
    for i in range(steps):
        ith_angle = 360 / divisor * (segment + i / steps)
        angles.append(ith_angle)
    arc_points = []
    for i, theta in enumerate(angles):
        SPATULA_ANGLE = 30 + 20*i/len(angles)

        
        # SPATULA CHARACTERISTICS
        SPATULA_RADIUS = 0.055 * np.cos(np.deg2rad(SPATULA_ANGLE))
        #SPATULA_OFFSET = 0.12
        EFFECTIVE_RADIUS = CAKE_RADIUS + SPATULA_RADIUS
        
        radius=EFFECTIVE_RADIUS
        x = center[0] + (radius) * np.cos(np.deg2rad(theta))
        y = center[1] + (radius) * np.sin(np.deg2rad(theta))
        if i == 0:
            x = center[0] + (radius) * np.cos(np.deg2rad(theta)) * 1.3
            y = center[1] + (radius) * np.sin(np.deg2rad(theta)) * 1.3
        elif i == (len(angles) - 1):
            x = center[0] + (radius) * np.cos(np.deg2rad(theta)) * 1.3
            y = center[1] + (radius) * np.sin(np.deg2rad(theta)) * 1.3

        arc_points.append(
            [x, y, LOWEST_Z, 0, 0.1, angle_clean(theta + 270 + SPATULA_ANGLE)]
        )

    start = list(arc_points[0])
    start[2] = start[2] + 0.1
    end = list(arc_points[-1])
    end[2] = end[2] + 0.1
    robot.move_global([start], time_step=0.6)
    robot.move_global(arc_points, time_step=(1, 0.04))
    robot.move_global([end], time_step=0.6)

def top_of_cake(joints_position, dx, dy):
    robot.move_joints([joints_position], time_step = 2)
    robot.move_global_r([[0.0,  dy, 0.0, 0.0, 0.0, 0.0],
                         [0.0, 0.0, 0.03, 0.0, 0.0, 0.0],
                         [-dx, -dy, -0.03, 0.0, 0.0, 0.0],
                         [0.0,  dy, 0.0, 0.0, 0.0, 0.0]], time_step = 0.8)

In [16]:
#robot.move_joints('home_to_cake', time_step = (2, 0.05), interp = 'spline')

arc(3, segment = 1)

[INFO] [1722457973.165687325] [remote_control_client]: Goal #48: Executing
[INFO] [1722457975.217250505] [remote_control_client]: Goal #48: Completed
[INFO] [1722457975.649856896] [remote_control_client]: Goal #49: Executing
[INFO] [1722457984.652099659] [remote_control_client]: Goal #49: Completed
[INFO] [1722457985.043466336] [remote_control_client]: Goal #50: Executing
[INFO] [1722457987.095496499] [remote_control_client]: Goal #50: Completed


In [26]:
ABOVE_CAKE_H = [-0.065, 0.301, 0.067, 50.071, 87.266, -100.51658272207979]
ABOVE_FROSTING_BEGINNING = [0.294, -0.102, 0.061, -29.745, 85.969, -87.3995992085437]


robot.move_joints('home_to_frosting_beginning', time_step = (2, 0.05), interp = 'spline')
robot.move_global([ABOVE_FROSTING_BEGINNING], time_step = 1)
#robot.move_global_r([[0.0, 0.0, -0.002, 0.0, 0.0, 0.0]], time_step = 0.3)
robot.move_global_r([[0.0, 0.13, 0.0, 0.0, 0.0, 0.0]], time_step = 4)
robot.move_joints('frosting_end_to_cake', time_step = (1, 0.05), interp = 'spline')

SEGMENTS = 3
for i in range(SEGMENTS):
    arc(SEGMENTS, segment = i)
    robot.move_joints('cake_to_frosting_beginning', time_step = (2, 0.05), interp = 'spline')
    robot.move_global([ABOVE_FROSTING_BEGINNING], time_step = 1)
    robot.move_global_r([[0.0, 0.0, -0.001 * (i + 1), 0.0, 0.0, 0.0]], time_step = 0.3)
    robot.move_global_r([[0.0, 0.13, 0.0, 0.0, 0.0, 0.0]], time_step = 4)
    robot.move_joints('frosting_end_to_cake', time_step = (1, 0.05), interp = 'spline')

robot.move_joints('cake_v_to_cake_h', time_step = (1, 0.04), interp = 'spline')
robot.move_global([ABOVE_CAKE_H], time_step = 1)
robot.move_global_r([[0.0, 0.09, 0.0, 0.0, 0.0, 0.0]], time_step = 3)

[INFO] [1722530073.201855483] [remote_control_client]: Goal #548: Executing
[INFO] [1722530080.255075109] [remote_control_client]: Goal #548: Completed
[INFO] [1722530082.778259226] [remote_control_client]: Goal #549: Executing
[INFO] [1722530083.830747637] [remote_control_client]: Goal #549: Completed
[INFO] [1722530086.348853664] [remote_control_client]: Goal #550: Executing
[INFO] [1722530090.401003116] [remote_control_client]: Goal #550: Completed
[INFO] [1722530090.406076322] [remote_control_client]: Goal #551: Executing
[INFO] [1722530096.459778777] [remote_control_client]: Goal #551: Completed
[INFO] [1722530098.812285023] [remote_control_client]: Goal #552: Executing
[INFO] [1722530099.464404013] [remote_control_client]: Goal #552: Completed
[INFO] [1722530102.052207087] [remote_control_client]: Goal #553: Executing
[INFO] [1722530107.105335178] [remote_control_client]: Goal #553: Completed
[INFO] [1722530109.655751612] [remote_control_client]: Goal #554: Executing
[INFO] [1722

KeyboardInterrupt: 

In [18]:
robot.move_joints('cake_h_to_cake_v', time_step = (1, 0.05), interp = 'spline')

[INFO] [1722458220.810389840] [remote_control_client]: Goal #82: Executing
[INFO] [1722458226.864321675] [remote_control_client]: Goal #82: Completed


In [25]:
arc(1, segment = 0)

[INFO] [1722527692.522385386] [remote_control_client]: Goal #545: Executing
[INFO] [1722527693.174064107] [remote_control_client]: Goal #545: Completed
[INFO] [1722527695.744138953] [remote_control_client]: Goal #546: Executing
[INFO] [1722527700.797420240] [remote_control_client]: Goal #546: Completed
[INFO] [1722527703.290769470] [remote_control_client]: Goal #547: Executing
[INFO] [1722527703.894133302] [remote_control_client]: Goal #547: Completed


In [11]:
robot.move_joints('home_to_frosting_beginning', time_step = (2, 0.08), interp = 'spline')

[INFO] [1722453035.098542297] [remote_control_client]: Goal #26: Executing
[INFO] [1722453045.152535664] [remote_control_client]: Goal #26: Completed


In [8]:
robot.move_global_r([[0.0, -0.13, 0.0, 0.0, 0.0, 0.0]], time_step = 6)

[INFO] [1722452586.618153554] [remote_control_client]: Goal #24: Executing
[INFO] [1722452592.620788174] [remote_control_client]: Goal #24: Completed


In [ ]:
robot.move_joints('cake_v_to_cake_h', time_step = (2, 0.08), interp = 'spline')
#robot.move_joints_r([[0.0, 0.0, 0.0, 0.0, 0.0, 2*np.pi]]) # this unfurs the wrist


In [ ]:
robot.move_global_r([[0.0, 0.13, 0.0, 0.0, 0.0, 0.0]], time_step = 6)

In [13]:
robot.read_global_pos()
[0.294, -0.102, 0.066, -29.745, 85.969, -87.3995992085437]

[0.29440462589263916,
 -0.10241859406232834,
 0.06636331230401993,
 -29.744996134920342,
 85.96923087054672,
 -87.3995992085437]

In [ ]:
#the robot starts at home
robot.move_joints('home_to_cake', time_step = (2, 0.08), interp = 'spline')
robot.move_joints('cake_to_frosting_beginning', time_step = (1, 0.08), interp = 'spline')
robot.move_global_r([[0, 0.15, 0, 0, 0, 0]])
robot.move_joints('frosting_end_to_cake', time_step = (1, 0.06), interp = 'spline')
arc(2, segment = 0)
robot.move_joints('cake_to_frosting_beginning', time_step = (2, 0.08), interp = 'spline')
robot.move_global_r([[0, 0.15, 0, 0, 0, 0]])
robot.move_joints('frosting_end_to_cake', time_step = (1, 0.06), interp = 'spline')
arc(2, segment = 1)
robot.move_joints('cake_to_frosting_beginning', time_step = (3, 0.08), interp = 'spline')
robot.move_global_r([[0, 0.15, 0, 0, 0, 0]])

In [ ]:
robot.move_global([CAKE_CENTER+[0, 0, 0]], time_step = 3)

In [ ]:
robot.read_joints_pos()
home_pos = [0, -90, 0, -90, 0, 0]

In [25]:
robot.move_joints_r([[0, 0, 0, 0, 0, 360]]) #to unfurl/unwind the robot

[INFO] [1722458701.778694588] [remote_control_client]: Goal #96: Executing


KeyboardInterrupt: 

In [ ]:
robot.move_joints('frosting_end_to_cake', time_step = (1, 0.08), interp = 'spline')

In [ ]:
robot.move_joints('cake_to_frosting_beginning', time_step = (3, 0.08), interp = 'spline')

In [ ]:
robot.read_joints_pos()

In [ ]:
robot.record('frosting_end_to_cake')

In [ ]:
arc(2, segment = 1)

In [6]:
robot.move_gripper(255, 255, 255) # close
#robot.move_gripper(0, 255, 255) # open